# Install library

In [5]:
%pip install urllib3
%pip install Pillow
%pip install opencv-python
%pip install requests
%pip install selenium
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


# Import library

In [6]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os

# Load config

In [7]:
load_dotenv('.env')

URL = os.getenv('URL')
URL_E = os.getenv('URL_E')

# Functions

In [8]:
# get book with image[0] have white border
def filter_book_image(url: str) -> bool:
    np.set_printoptions(threshold=sys.maxsize)

    response = requests.get(url)

    image = Image.open(BytesIO(response.content))
    arr = np.asarray(image)
    t = [[255,255,255]]*50
    
    return np.sum(arr[0][:50]) != np.sum(t)

In [9]:
# save to file csv
def save_file(name_file,header, data):
    with open(name_file+'.csv', 'w') as file:
        # 2. Create a CSV writer
        writer = csv.writer(file)
        # 3. Write data to the file
        writer.writerow(header)
        writer.writerows(data)

In [10]:
def get_reviews(driver):
    time.sleep(0.5)
    review = driver.find_elements(By.XPATH, '//ul[@class="comment_list"]//li')
    review_text = [re.text for re in review]
    review_end = [re.split("\n")[1:3] for re in review_text]
    
    return review_end

In [11]:
def get_data(driver):
    book_title = driver.find_element(By.XPATH, '//div[@class="product-essential-detail"]/h1').text
            
    pushlisher = driver.find_element(By.XPATH,'//div[@class="product-view-sa_two"]/div[@class="product-view-sa-supplier"]/span[2]').text
    author = driver.find_elements(By.XPATH,'//div[@class="product-view-sa-author"]//span[2]')[0].text
    
    price = driver.find_element(By.XPATH,'//div[@id="catalog-product-details-price"]//p[@class="old-price"]/span[@class="price"]').text
    if not price:
        price = driver.find_element(By.XPATH,'//div[@id="catalog-product-details-price"]//p[@class="special-price"]/span[@class="price"]').text
    
    category_list = driver.find_elements(By.XPATH,'//ol[@class="breadcrumb"]/li')
    categories = [cat.text for cat in category_list]
    category = [categories[1], categories[2]]
        
    rating = driver.find_element(By.XPATH,'//table[@class="ratings-desktop"]//div[@class="rating"]').get_attribute("style")
    
    image_list = driver.find_elements(By.XPATH,'//div[@id="lightgallery-product-media"]//a')
    images = [image.get_attribute("href") for image in image_list]
    
    desc = driver.find_elements(By.XPATH,'//div[@id="desc_content"]')[0].text
    description=desc.replace("\n","\\n")
    
    weight = driver.find_elements(By.XPATH,'//td[@class="data_weight"]')[0].text
    
    return [book_title,
            pushlisher,
            author,
            price,
            category,
            rating, 
            images,
            description,
            weight,]

# Get data

## Create driver chrome

In [12]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")

driver=webdriver.Chrome(options=chrome_options)

url = URL
driver.get(url)

## Get data raw

In [13]:
fields = ['book_title',
            'pushlisher',
            'author',
            'price',
            'category',
            'rating', 
            'images',
            'desc',
            'weight',
            'rating_count',
            'reviews']

pagination = driver.find_elements(By.XPATH, '//div[@id="pagination"]//a')
max_pag =  int(pagination[len(pagination)-2].text)
# max_pag =  100
# max_pag =  1
data = []

url = URL_E
for pag in range(1,max_pag+1):
    url_pag = url + str(pag)
    driver.get(url_pag)
    
    books = driver.find_elements(By.XPATH, '//div[@class="ma-box-content"]/h2[@class="product-name-no-ellipsis p-name-list"]/a')
    
    book_url = [book_url.get_attribute("href") for book_url in books]
    for e in book_url:
        try:
            driver.get(e)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(1)
            
            data_book = get_data(driver)
            pag_reviews = driver.find_elements(By.XPATH, '//div[@id="pagination"]//a')
             
            reviews=[]
            if len(pag_reviews) > 1:
                pag_re = int([re.text for re in pag_reviews][-2])
                for _ in range(pag_re-1):
                    wait = WebDriverWait(driver, 20)
                    review_end = get_reviews(driver)
                    
                    reviews.extend(review_end)
                    wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@id="pagination"]//li[@title="Next"]'))).click()
                    
                review_end = get_reviews(driver)
                reviews.extend(review_end)
            else:
                review_end = get_reviews(driver)
                reviews.extend(review_end)
            
            if filter_book_image(data_book[6][0]) and data_book[4][0] != 'OTHER LANGUAGES':
                data_book.append(len(reviews))
                data_book.append(reviews)
                data.append(data_book)
                
                save_file('data_raw/books_raw',fields,data)
        except Exception as e:
            print(e)
        
    if len(data) % 300 == 0:
        print(data)
        
    if len(data) >= 3000:
            break

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id="catalog-product-details-price"]//p[@class="old-price"]/span[@class="price"]"}
  (Session info: chrome=116.0.5845.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55c360f31e23 <unknown>
#1 0x55c360c5a7a7 <unknown>
#2 0x55c360c991d3 <unknown>
#3 0x55c360c992c1 <unknown>
#4 0x55c360cd4a04 <unknown>
#5 0x55c360cba03d <unknown>
#6 0x55c360cd2369 <unknown>
#7 0x55c360cb9de3 <unknown>
#8 0x55c360c8da7b <unknown>
#9 0x55c360c8e81e <unknown>
#10 0x55c360ef3638 <unknown>
#11 0x55c360ef7507 <unknown>
#12 0x55c360f01c4c <unknown>
#13 0x55c360ef8136 <unknown>
#14 0x55c360ec69cf <unknown>
#15 0x55c360f1bb98 <unknown>
#16 0x55c360f1bd68 <unknown>
#17 0x55c360f2acb3 <unknown>
#18 0x7f4584494b43 <unknown>

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@i

## Close driver

In [14]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.96)
Stacktrace:
#0 0x55c360f31e23 <unknown>
#1 0x55c360c5a7a7 <unknown>
#2 0x55c360c3f340 <unknown>
#3 0x55c360c4084f <unknown>
#4 0x55c360c407a4 <unknown>
#5 0x55c360c30ef1 <unknown>
#6 0x55c360c31cc0 <unknown>
#7 0x55c360cc4c42 <unknown>
#8 0x55c360cb9fd8 <unknown>
#9 0x55c360c8da7b <unknown>
#10 0x55c360c8e81e <unknown>
#11 0x55c360ef3638 <unknown>
#12 0x55c360ef7507 <unknown>
#13 0x55c360f01c4c <unknown>
#14 0x55c360ef8136 <unknown>
#15 0x55c360ec69cf <unknown>
#16 0x55c360f1bb98 <unknown>
#17 0x55c360f1bd68 <unknown>
#18 0x55c360f2acb3 <unknown>
#19 0x7f4584494b43 <unknown>
